In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
from collections import namedtuple

import gpflow
from gpflow.utilities import print_summary, positive

import tensorflow as tf
from tensorflow import math as tfm
from tensorflow_probability import bijectors as tfb
from tensorflow_probability import distributions as tfd
from tensorflow_probability import mcmc

from load_puma_data import load_barenco_puma
import math
import random

PI = tf.constant(math.pi, dtype='float64')
plt.style.use('ggplot')
%matplotlib inline

In [ ]:
df, genes, genes_se, Y, Y_var = load_barenco_puma()


N_m = 7               # Number of observations
def calc_N_p(N_p, num_disc=8):
    '''A helper recursive function to ensure t is a subset of τ'''
    if num_disc <= 0:
        return N_p
    return N_p -1 + calc_N_p(N_p, num_disc-1)
num_disc = 10
N_p = calc_N_p(N_m, num_disc)  # Number of time discretisations
common_indices = np.array([i*num_disc+i for i in range(N_m)])
t = np.arange(N_m)*2           # Observation times
τ = np.linspace(0, 12, N_p, dtype='float64')    # Discretised observation times
num_genes = 5
I = 1 # Number of TFs

m_observed = np.float64(Y[:-1])
f_observed = np.float64(np.atleast_2d(Y[-1]))
σ2 = np.float64(Y_var[:-1])
σ2_f = np.float64(np.atleast_2d(Y_var[-1]))

## Metropolis Hastings Custom MCMC Algorithm

In [ ]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.5f}".format(x)})

def get_rbf_dist(times, N):
    t_1 = np.reshape(np.tile(times, N), [N, N]).T
    t_2 = np.reshape(np.tile(times, N), [N, N])
    return t_1-t_2

def jitter_cholesky(A):
    try:
        jitter1 = tf.linalg.diag(1e-7 * np.ones(A.shape[0]))
        return tf.linalg.cholesky(A + jitter1)
    except:
        jitter2 = tf.linalg.diag(1e-5 * np.ones(A.shape[0]))
        return tf.linalg.cholesky(A + jitter2)

info = np.finfo('float64')
def exp(x):
    '''Safe exp'''
    with np.errstate(under='ignore', over='ignore'):
        return np.exp(x)

In [ ]:
np.seterr(all='raise')

class MetropolisHastings():
    '''MH accept function'''
    def is_accepted(self, new_log_prob, old_log_prob):
        alpha = exp(new_log_prob - old_log_prob)
        if tf.is_tensor(alpha):
            alpha = alpha.numpy()
        return not np.isnan(alpha) and random.random() < min(1, alpha)

class TranscriptionLikelihood():
    def predict_m(self, kbar, δbar, w, fbar, w_0):
        # Take relevant parameters out of log-space
        a_j, b_j, d_j, s_j = (np.exp(kbar[:, i]).reshape(-1, 1) for i in range(4))
        δ = np.exp(δbar)
        f_i = np.log(1+np.exp(fbar))
    #     print('f_i', f_i)

        # Calculate p_i vector
        p_i = np.zeros(N_p) # TODO it seems the ODE translation model has params A, S see gpmtfComputeTFODE
        Δ = τ[1]-τ[0]
        sum_term = exp(δ*τ) * f_i
        p_i[1:] = 0.5*Δ*np.cumsum(sum_term[:-1] + sum_term[1:]) # Trapezoid rule
        p_i = exp(-δ*τ) * p_i
    #     print('pi', p_i)

        # Calculate m_pred
        integrals = np.zeros((num_genes, N_p))
        interactions = w[:, 0][:, None]*np.log(p_i+1e-50) + w_0    
        G = 1/(1+np.exp(-interactions)) # TF Activation Function
        sum_term = G * np.exp(d_j*τ)
        integrals[:, 1:] = 0.5*Δ*np.cumsum(sum_term[:, :-1] + sum_term[:, 1:], axis=1) # Trapezoid rule
        exp_dt = np.exp(-d_j*τ)
        integrals = exp_dt * integrals
        m_pred = b_j/d_j + (a_j-b_j/d_j)*exp_dt + s_j*integrals

        return m_pred

    def genes(self, params, δbar=None, 
                     fbar=None, 
                     kbar=None, 
                     w=None,
                     σ2_m=None):
        '''
        Computes likelihood of the genes.
        If any of the optional args are None, they are replaced by their current value in params.
        '''
        if δbar is None:
            δbar = params.δbar.value
        if fbar is None:
            fbar = params.fbar.value
        if kbar is None:
            kbar = params.kbar.value
        w = params.w.value if w is None else w
        σ2_m = params.σ2_m.value if σ2_m is None else σ2_m

        w_0 = 0 # TODO no hardcode this!
        m_pred = self.predict_m(kbar, δbar, w, fbar, w_0)

        log_lik = np.zeros(num_genes)
        sq_diff = np.square(m_observed - m_pred[:, common_indices])
        variance = σ2_m.reshape(-1, 1) + σ2 # add PUMA variance
        log_lik = -0.5*np.log(2*np.pi*(variance)) - 0.5*sq_diff/variance
        log_lik = np.sum(log_lik, axis=1)

        return log_lik

    def tfs(self, fbar, i=0): 
        '''
        Computes log-likelihood of the transcription factors.
        TODO this should be for the i-th TF
        '''
        f_pred = np.log(1+np.exp(fbar))
        f_pred = np.float64(np.atleast_2d(f_pred[common_indices]))
        sq_diff = np.square(f_observed[i] - f_pred[i])
#         prob = tfd.Normal(f_observed[i], σ2_f[i]).log_prob(f_i[i])
        log_lik = -0.5*sum(np.log(2*np.pi*σ2_f[i])) - 0.5*sum(sq_diff/σ2_f[i])
        return log_lik

class Parameter():
    def __init__(self, prior, initial_value, proposal_dist=None, name=None, constraint=None):
        self.name = name
        self.prior = prior
        self.proposal_dist = proposal_dist
        if constraint is None:
            self.constrained = lambda x:x
        else:
            self.constrained = constraint
        self.value = initial_value

    def set_value(self, value):
        this.value = value
    def constrain(self, *args):
        return self.constrained(*args)
    
    def propose(self, *args):
        assert self.proposal_dist is not None, 'proposal_dist must not be None if you use propose()'
        return self.proposal_dist(*args).sample().numpy()


In [ ]:
f64 = np.float64
class TranscriptionMCMC(MetropolisHastings):
    def __init__(self):
        self.likelihood = TranscriptionLikelihood()
        self.clear_samples()
        # Adaptable variances
        a = tf.constant(-0.5, dtype='float64')
        b2 = tf.constant(2., dtype='float64')
        self.h_δ = 0.15
        h_c = 0.1
        self.h_f = 0.4*tf.ones(N_p, dtype='float64')
        self.h_k = 0.15*tf.ones(4, dtype='float64')
        h_w = tf.ones(num_genes, dtype='float64')
        h_σm = tf.constant(0.1, dtype='float64')

        # Interaction weights
        w_j0 = 0   
        w = Parameter(
            tfd.Normal(0, 2), 
            0.1*np.ones((num_genes, I)),
            proposal_dist=lambda mu:tfd.Normal(mu, 0.1)) #h_w[j]) w_j) # At the moment this is the same as w_j0 (see pg.8)
        # Latent function
        fbar = Parameter(
            self.fbar_prior, 
            0.5*np.ones(N_p))
        # GP hyperparameters
        V = Parameter(tfd.InverseGamma(f64(0.1), f64(0.1)), f64(1), 
                      proposal_dist=lambda v: tfd.TruncatedNormal(v, h_c, low=0, high=100)) #v_i Fix to 1 if translation model is not used (pg.8)
        L = Parameter(tfd.InverseGamma(f64(0.1), f64(0.1)), f64(0.1),
                      proposal_dist=lambda l2: tfd.TruncatedNormal(l2, h_c, low=0, high=100)) #l2_i
        # Translation kinetic parameters
        δbar = Parameter(tfd.Normal(a, b2), f64(-0.3), proposal_dist=lambda mu:tfd.Normal(mu, self.h_δ))
        # White noise for genes
        σ2_m = Parameter(tfd.InverseGamma(f64(0.01), f64(0.01)), 1e-2*np.ones(num_genes),
                         proposal_dist=lambda mu: tfd.TruncatedNormal(mu, h_σm, low=0, high=5))
        # Transcription kinetic parameters
        def constrain_kbar(kbar, gene):
            '''Constrains a given row in kbar'''
            if gene == 3:
                kbar[2] = np.log(0.8)
                kbar[3] = np.log(1.0)
            kbar[kbar < -10] = -10
            kbar[kbar > 2] = 2
            return kbar
        kbar_initial = -0.1*np.float64(np.c_[
            np.ones(num_genes), # a_j
            np.ones(num_genes), # b_j
            np.ones(num_genes), # d_j
            np.ones(num_genes)  # s_j
        ])
        for j, k in enumerate(kbar_initial):
            kbar_initial[j] = constrain_kbar(k, j)
        kbar = Parameter(
            tfd.Normal(a, b2), 
            kbar_initial,
            proposal_dist=lambda mu: tfd.MultivariateNormalDiag(mu, self.h_k),
            constraint=constrain_kbar)
        params = namedtuple('parameters', ['fbar','δbar','kbar','σ2_m','w','L','V'])
        self.params = params(fbar, δbar, kbar, σ2_m, w, L, V)
        
    def fbar_prior_params(self, v, l2):
        t_dist = get_rbf_dist(τ, N_p)
    #     print('vl2', v, l2)
        jitter = tf.linalg.diag(1e-5 * np.ones(N_p))
        try:
            K = v * exp(-np.square(t_dist)/(2*l2)) + jitter
        except:
#             print('HELP! sq', -np.square(t_dist)/(2*l2))
#             raise 'Error!'
            K = np.eye(N_p)

    #     print(K)
        m = np.zeros(N_p)
        return m, K

    def fbar_prior(self, fbar, v, l2):
        m, K = self.fbar_prior_params(v, l2)

        try:
            return tfd.MultivariateNormalFullCovariance(m, K).log_prob(fbar)
        except:
            jitter = tf.linalg.diag(1e-4 * np.ones(N_p))
            try:
                return np.float64(tfd.MultivariateNormalFullCovariance(m, K+jitter).log_prob(fbar))
            except:
                return 0

    def clear_samples(self):
        self.samples = {
            'δbar': list(), 
            'kbar': [list() for _ in range(num_genes)],
            'σ2_m': [list() for _ in range(num_genes)],
            'w': [list() for _ in range(num_genes)],
            'L': list(),
            'V': list(),
            'fbar': list(),
            'acc_rates': list()
        }
        
    def adapt(self):
        T = 5
        plt.figure(figsize=(10,10))
        for i, h_δ_test in enumerate(np.linspace(0.1, 0.25, T)):
            self.clear_samples()
            self.h_δ = h_δ_test
            plt.subplot(T, 3, i+1)
            self.sample(T=300, store_every=1, burn_in=0, report_every=200) # sample, do not report
            plt.acorr(self.samples['δbar'], maxlags=50)
            plt.tight_layout()
            plt.xlim(-1, 50)
            plt.title(f'h_δ = {h_δ_test:3f}')
            print(self.acceptance_rates)
        
        # Selecting...
        self.h_δ = 0.30
        self.clear_samples()
        self.sample(T=1000, store_every=1, burn_in=0)
#         plt.plot(self.samples['acceptance_rates']['δ'])
    
    def sample(self, T=20000, store_every=10, burn_in=1000, report_every=100):
        print('----- Metropolis Begins -----')
        self.acceptance_rates = { # Reset acceptance rates
            'f': 0.,
            'δ': 0.,
            'k': 0.,
            'σ': 0.,
            'w': 0.,
            'L': 0.,
            'V': 0.
        }

        for iteration_number in range(T):
            if iteration_number % report_every == 0:
                print(f'{100*iteration_number/T:.2f}% complete', end='\r')

            self.iterate()
    
            if iteration_number >= burn_in and iteration_number % store_every == 0:
                for j in range(num_genes):
                    self.samples['σ2_m'][j].append(self.params.σ2_m.value[j].copy())
                    self.samples['w'][j].append(self.params.w.value[j].copy())
                    self.samples['kbar'][j].append(self.params.kbar.value[j].copy())

                self.samples['V'].append(self.params.V.value)
                self.samples['L'].append(self.params.L.value)
                self.samples['δbar'].append(self.params.δbar.value)
                self.samples['fbar'].append(self.params.fbar.value)
                self.samples['acc_rates'].append(list(self.acceptance_rates.values()))
                
        for key in self.acceptance_rates:
            self.acceptance_rates[key] /= T
        print('----- Finished -----')

    def iterate(self):
        params = self.params
        # Compute likelihood for comparison
        old_m_likelihood = self.likelihood.genes(params)
        
        # Untransformed tf mRNA vectors F
        fbar = params.fbar.value
        for i in range(I):
            # Gibbs step
            z_i = tf.reshape(tfd.MultivariateNormalDiag(fbar, self.h_f).sample(), (1, -1))
            # MH
            m, K = self.fbar_prior_params(params.V.value, params.L.value)
            invKsigmaK = tf.matmul(tf.linalg.inv(K+tf.linalg.diag(self.h_f)), K) # (C_i + hI)C_i
            L = jitter_cholesky(K-tf.matmul(K, invKsigmaK))
            c_mu = tf.matmul(z_i, invKsigmaK)
            fstar = tf.matmul(tf.random.normal((1, L.shape[0]), dtype='float64'), L) + c_mu
            fstar = tf.reshape(fstar, (-1, ))
            
            new_prob = np.sum(self.likelihood.genes(params, fbar=fstar)) + self.likelihood.tfs(fstar)
            old_prob = np.sum(old_m_likelihood) + self.likelihood.tfs(fbar)
            if self.is_accepted(new_prob, old_prob):
                params.fbar.value = fstar
                self.acceptance_rates['f'] += 1/I


        # Log of translation ODE degradation rates
        δbar = params.δbar.value
        for i in range(I):# TODO make for I > 1
            # Proposal distribution
            δstar = params.δbar.propose(δbar) # δstar is in log-space, i.e. δstar = δbar*
            new_prob = np.sum(self.likelihood.genes(params, δbar=δstar)) + params.δbar.prior.log_prob(δstar)
            old_prob = np.sum(old_m_likelihood) + params.δbar.prior.log_prob(δbar)
#             print(δstar, params.δbar.prior.log_prob(δstar))
#             print(new_prob, old_prob)
            if self.is_accepted(new_prob, old_prob):
                params.δbar.value = δstar
                self.acceptance_rates['δ'] += 1/I

        # Log of transcription ODE kinetic params
        kbar = params.kbar.value
        kstar = kbar.copy()
        for j in range(num_genes):
            sample = params.kbar.propose(kstar[j])
            sample = params.kbar.constrain(sample, j)

            kstar[j] = sample
            new_prob = self.likelihood.genes(params, kbar=kstar)[j] + sum(params.kbar.prior.log_prob(sample))
            old_prob = old_m_likelihood[j] + sum(params.kbar.prior.log_prob(kbar[j]))
            if self.is_accepted(new_prob, old_prob):
                test = params.kbar.value
                test[j]=sample
                params.kbar.value = test
                self.acceptance_rates['k'] += 1/num_genes
            else:
                kstar[j] = params.kbar.value[j]


        # Interaction weights and biases (note: should work for I > 1)
        w = params.w.value
        wstar = w.copy()
        for j in range(num_genes):
            sample = params.w.propose(wstar[j])
            wstar[j] = sample
            new_prob = self.likelihood.genes(params, w=wstar)[j] + sum(params.w.prior.log_prob(sample))
            old_prob = old_m_likelihood[j] + sum(params.w.prior.log_prob(w[j,:]))
            if self.is_accepted(new_prob, old_prob):
                params.w.value[j] = sample
                self.acceptance_rates['w'] += 1/num_genes
            else:
                wstar[j] = params.w.value[j]

        # Noise variances
        σ2_m = params.σ2_m.value
        σ2_mstar = σ2_m.copy()
        for j in range(num_genes):
            sample = params.σ2_m.propose(σ2_m[j])
            σ2_mstar[j] = sample
            old_q = params.σ2_m.proposal_dist(σ2_mstar[j]).log_prob(σ2_m[j])
            new_prob = self.likelihood.genes(params, σ2_m=σ2_mstar)[j] +params.σ2_m.prior.log_prob(σ2_mstar[j])
            
            new_q = params.σ2_m.proposal_dist(σ2_m[j]).log_prob(σ2_mstar[j])
            old_prob = self.likelihood.genes(params, σ2_m=σ2_m)[j] + params.σ2_m.prior.log_prob(σ2_m[j])
                
            if self.is_accepted(new_prob + old_q, old_prob + new_q):
                params.σ2_m.value[j] = sample
                self.acceptance_rates['σ'] += 1/num_genes
            else:
                σ2_mstar[j] = σ2_m[j]

        # Length scales and variances of GP kernels
        l2 = params.L.value
        v = params.V.value
        for i in range(I):
            # Proposal distributions
            Q_v = params.V.proposal_dist
            Q_l = params.L.proposal_dist
            vstar = params.V.propose(v)
            l2star = params.L.propose(l2)
#             print(vstar, l2star)# 'prior', fbar_prior(fbar_i, vstar, l2star))
            # Acceptance probabilities
            old_q = Q_v(vstar).log_prob(v) + Q_l(l2star).log_prob(l2) # Q(old|new)
            new_prob = params.fbar.prior(params.fbar.value, vstar, l2star) + params.V.prior.log_prob(vstar) + params.L.prior.log_prob(l2star)

            new_q = Q_v(v).log_prob(vstar) + Q_l(l2).log_prob(l2star) # Q(new|old)
            old_prob = params.fbar.prior(params.fbar.value, v, l2) + params.V.prior.log_prob(v) + params.L.prior.log_prob(l2)

            accepted = self.is_accepted(new_prob + old_q, old_prob + new_q)
            if accepted:
#                 print("accepted")
                params.V.value = vstar
                params.L.value = l2star
                self.acceptance_rates['V'] += 1/I
                self.acceptance_rates['L'] += 1/I

    def predict_m(self):
        return self.likelihood.predict_m(self.params.kbar.value, 
                                         self.params.δbar.value, 
                                         self.params.w.value, 
                                         self.params.fbar.value, 0)

transcription_model = TranscriptionMCMC()


In [ ]:
# Begin MCMC Adaptive phase
transcription_model.adapt()

In [ ]:
# Begin MCMC
T = 1000
store_every = 20
burn_in = 100
report_every = 50

transcription_model.sample(T, store_every, burn_in, report_every)

print(transcription_model.acceptance_rates)

samples = transcription_model.samples


Step size is standard dev, too small means it takes long time to reach high density areas. too long means we reject many of samples

## Plots

In [ ]:
samples = transcription_model.samples

plt.figure(figsize=(10,14))

parameter_names = transcription_model.acceptance_rates.keys()
acc_rates = np.array(samples['acc_rates'])
print(T/(acc_rates.shape[0]+1))
for i, name in enumerate(parameter_names):
    plt.subplot(len(parameter_names), 3, i+1)
    deltas = acc_rates[:, i]/np.arange(1, T-burn_in, store_every)
    plt.plot(deltas)
    plt.title(name)
plt.tight_layout()

In [ ]:
# Plot decay
plt.figure(figsize=(10, 8))
for i, param in enumerate(['δbar', 'L', 'V']):
    ax = plt.subplot(331+i)
    plt.plot(samples[param])
    ax.set_title(param)
#'σ', 'w']):

plt.figure()
for j in range(num_genes):
    plt.plot(samples['w'][j])
    
plt.title('Interaction weights')

In [ ]:
# Plot transcription ODE kinetic params
plt.figure(figsize=(10, 14))
plt.title('Transcription ODE kinetic parameters')
labels = ['a', 'b', 'd', 's']
for j in range(num_genes):
    ax = plt.subplot(num_genes, 2, j+1)
    k_param = np.array(samples['kbar'][j])
#     print(k_param)
    
    for k in range(4):
        plt.plot(k_param[:, k], label=labels[k])
    plt.legend()
    ax.set_title(f'Gene {j}')

plt.tight_layout()

plt.figure(figsize=(14, 14))
k_latest = np.exp(np.array([
    np.mean(np.array(samples['kbar'][j])[-10:], axis=0) for j in range(num_genes)]))
print(k_latest)
B = k_latest[:,1]
D = k_latest[:,2]
S = k_latest[:,3]
print(B)
B_barenco = np.array([2.6, 1.5, 0.5, 0.2, 1.35])# From Martino paper ... but don't know the scale
B_barenco = B_barenco/np.mean(B_barenco)*np.mean(B)# do a rough rescaling so that the scales match.
S_barenco = np.array([3, 0.8, 0.7, 1.8, 0.7])/1.8
D_barenco = np.array([1.2, 1.6, 1.75, 3.2, 2.3])*0.8/3.2


data = [B, S, D]
barenco_data = [B_barenco, S_barenco, D_barenco]
labels = ['Basal rates', 'Sensitivities', 'Decay rates']

plotnum = 331
for A, B, label in zip(data, barenco_data, labels):
    plt.subplot(plotnum)
    plotnum+=1
    plt.bar(np.arange(5)-0.2, A, width=0.4, tick_label=df.index[:-1])
    plt.bar(np.arange(5)+0.2, B, width=0.4, color='blue', align='center')

    plt.title(label)


In [ ]:
# Plot genes
plt.figure(figsize=(10, 10))
m_pred = transcription_model.predict_m()
print(m_pred.shape)
for j in range(num_genes):
    plt.subplot(531+j)
    plt.plot(m_pred[j, :])

In [ ]:
plt.figure(figsize=(12, 10))
plt.title('Noise variances')
for i, j in enumerate(range(num_genes)):
    ax = plt.subplot(num_genes, num_genes-2, i+1)
    plt.plot(samples['σ2_m'][j]);

In [ ]:
def plot_f():
    f_i = np.log(1+np.exp(transcription_model.params.fbar.value))
    plt.plot(f_i)
    plt.figure()
    f = f_i[common_indices]
    f[0] = 0
    scale_pred = np.sqrt(np.var(f));
    barencof = np.array([[0.0, 200.52011, 355.5216125, 205.7574913, 135.0911372, 145.1080997, 130.7046969],
                         [0.0, 184.0994134, 308.47592, 232.1775328, 153.6595161, 85.7272235, 168.0910562],
                         [0.0, 230.2262511, 337.5994811, 276.941654, 164.5044287, 127.8653452, 173.6112139]])

    barencof = barencof[0]/(np.sqrt(np.var(barencof[0])))*scale_pred;
    measured_p53 = df[df.index.isin(['211300_s_at', '201746_at'])]
    print(measured_p53)
    measured_p53 = measured_p53.mean(0)
    measured_p53 = measured_p53*scale_pred
    lb = len(barencof)
    print(lb)
    plt.plot(np.arange(lb), f)
    plt.scatter(np.arange(lb), barencof, marker='x')


In [ ]:
plot_f()

In [ ]:
transcription_model.sample(T=1)
plot_f()

In [ ]:
print(samples['σ'])

In [ ]:
fmean = (np.mean(samples['fbar'][-60::10], axis=0))
plt.plot(fmean[common_indices])

plt.figure()
f = np.array(samples['fbar'])
for i in common_indices:
    plt.plot(f[:, i])